# Team

<a target="_blank" href="https://colab.research.google.com/drive/1iuP5kw92Qa48QMUJW-K50r-pSouXYgD-?usp=sharing"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Helder Mateus dos Reis Matos

# Attention, please!

Solve the problems below. Show the problem model and the solution using ```pyomo``` and the GLPK or COIN-OR solvers.

To see how to write optimization models in colab, look for examples in the Linear Programming class colab.

Please send your Colab file (.ipynb) in SIGAA.

In [ ]:
# !pip install -q pyomo
# !apt install -q -y glpk-utils

In [1]:
import pyomo.environ as pyo

# Problem 1

FFB fresh fruit business mixes apples, peaches, and nectarines to make three different types of baskets for local market. Each basket contains approximately 5 kg fruits. The content of three types of baskets is specified as follows:

|Basket Type|Apple|Peach|Nectarine|
|--|--|--|--|
|1|At least 30%|At most 20%|-|
|2|-|At most 40%|At least 20%|
|3|At least 20%|-|At most 30%|

FFB purchases apple at a cost of $1.00/kg, peach $1.50/kg, and nectarine $1.80/kg, and sells type-1 basket at $2.25/kg, type-2 basket $3.00/kg, and type-3 basket $2.60/kg. The daily supply of fruit is limited to 60 kg of apples, 70 kg of peaches, and 50 kg of nectarines.  FFB is able to sell all the fruit baskets they prepare for a given day.

Formulate an ILP model to determine how the fruit be mixed in order to maximize the profit. Solve it.

## 1.1. Objective function

The objective is to maximize profit, defined as the difference between the total revenue from selling fruit baskets and the total cost of purchasing fruits:

$$\max Z = 2.25(5y_{1}) + 3.00(5y_{2}) + 2.60(5y_{3}) - (1.00\sum a_{j} + 1.50\sum p_{j} + 1.80\sum n_{j})$$

Where:

- $y_j$: number of baskets of type $j$ (1, 2 or 3)
- $a_j, p_j, n_j$: kilograms of apples, peaches, and nectarines in basket type $j$

## 1.2. Constraints

Each basket must weigh 5kg and follow the composition requirements:

- Type 1: Apple $\geq$ 30%, Peach $\leq$ 20%
- Type 2: Peach $\leq$ 40%, Nectarine $\geq$ 20%
- Type 3: Apple $\geq$ 20%, Nectarine $\leq$ 30%

Supply and non-negativity constraints:

$$a_1 + a_2 + a_3 \leq 60$$

$$p_1 + p_2 + p_3 \leq 70$$

$$n_1 + n_2 + n_3 \leq 50$$

## 1.3. PyOMO model

A `ConcreteModel()` is created to store information regarding the problem. Next, the list `types` represents the three basket types and is converted into a Pyomo set so that all constraints and equations can be written in a compact, indexed form.

In [2]:
model_basket = pyo.ConcreteModel()

types = [1, 2, 3]
model_basket.types = pyo.Set(initialize=types)

The decision variables correspond directly to production and composition choices. The integer variable `y` indicates how many baskets of each type will be assembled, while the continuos variables `a`, `p`, and `n` represent the kilograms of apples, peaches, and nectarines used in each basket type. Keeping the weight variables continuous allows the optimizer to freely adjust the fruit proportions within the allowed composition limits.

In [3]:
model_basket.y = pyo.Var(model_basket.types, domain=pyo.NonNegativeIntegers)
model_basket.a = pyo.Var(model_basket.types, domain=pyo.NonNegativeReals)
model_basket.p = pyo.Var(model_basket.types, domain=pyo.NonNegativeReals)
model_basket.n = pyo.Var(model_basket.types, domain=pyo.NonNegativeReals)

The variable `basket_mass` establishes that each basket weights 5 kilograms.

The dictionary `supply` specifies the daily availability of each fruit type, in kilograms. Next, the dictionaries `cost` and `price_per_kg` represent the economic aspects of the problem.

The `cost` dictionary gives the purchase price of each fruit per kilogram, while `price_per_kg` indicates the selling price of each basket type, also expressed per kilogram of the final mix.

In [4]:
basket_mass = 5.0
supply = {"a": 60.0, "p": 70.0, "n": 50.0}
cost = {"a": 1.00, "p": 1.50, "n": 1.80}
price_per_kg = {1: 2.25, 2: 3.00, 3: 2.60}

As for the constraints, we need to ensure consistency between the total fruit quantities used and the number of baskets produced. The function `mass_rule(m, j)` returns an equality that enforces, for each basket type `j`, the sum of apples, peaches, and nectarines (`m.a[j] + m.p[j] + m.n[j]`) to be exactly equal to the total mass of baskets of that type (`basket_mass * m.y[j]`).

This constraint is automatically applied to every basket type in the set. This formulation guarantees that no matter how the optimizer chooses to combine fruits, each basket always respects the 5 kg total weight limit.


In [5]:
def mass_rule(m, j):
    return m.a[j] + m.p[j] + m.n[j] == basket_mass * m.y[j]

model_basket.mass = pyo.Constraint(model_basket.types, rule=mass_rule)

For each type of fruit, upper and lower bounds are applied based on the percentage of each fruit within every basket type.

In [6]:
model_basket.types1_a_min = pyo.Constraint(expr=model_basket.a[1] >= 0.30 * basket_mass * model_basket.y[1])
model_basket.types1_p_max = pyo.Constraint(expr=model_basket.p[1] <= 0.20 * basket_mass * model_basket.y[1])

model_basket.types2_p_max = pyo.Constraint(expr=model_basket.p[2] <= 0.40 * basket_mass * model_basket.y[2])
model_basket.types2_n_min = pyo.Constraint(expr=model_basket.n[2] >= 0.20 * basket_mass * model_basket.y[2])

model_basket.types3_a_min = pyo.Constraint(expr=model_basket.a[3] >= 0.20 * basket_mass * model_basket.y[3])
model_basket.types3_n_max = pyo.Constraint(expr=model_basket.n[3] <= 0.30 * basket_mass * model_basket.y[3])

Now it is going to be ensured that the total amount of each fruit used across all basket types does not exceed the available daily stock. The model sums the kilograms of apples, peaches, and nectarines used in all types and compares each total to its respective limit in the `supply` dictionary. This prevents the model from allocating more fruit than what is available in storage.

In [7]:
model_basket.apple_sup  = pyo.Constraint(expr=sum(model_basket.a[j] for j in model_basket.types) <= supply["a"])
model_basket.peach_sup  = pyo.Constraint(expr=sum(model_basket.p[j] for j in model_basket.types) <= supply["p"])
model_basket.nec_sup    = pyo.Constraint(expr=sum(model_basket.n[j] for j in model_basket.types) <= supply["n"])

Lastly, the objective function aims to maximize the company’s daily profit. The variable `revenue` represents the total value from selling baskets, while the next three expressions compute the total cost of purchasing apples, peaches, and nectarines, aggregated in `fruit_cost`.

Finally, the objective function is declared as `revenue - fruit_cost` with `sense=pyo.maximize`, meaning that the solver will search for the production combination with the highest profit.

In [8]:
revenue = sum(price_per_kg[j] * basket_mass * model_basket.y[j] for j in model_basket.types)

cost_apples = sum(model_basket.a[j] for j in model_basket.types) * cost["a"]
cost_peaches = sum(model_basket.p[j] for j in model_basket.types) * cost["p"]
cost_nectarines = sum(model_basket.n[j] for j in model_basket.types) * cost["n"]

fruit_cost = cost_apples + cost_peaches + cost_nectarines

model_basket.obj = pyo.Objective(expr=revenue - fruit_cost, sense=pyo.maximize)

## 1.4. Solving with GLPK

The maximum profit obtained is \$285, where the best combination concentrates production entirely in Type-2 baskets, with `y[2] = 36` baskets (180 kg). The model uses all available supplies `a[2] = 60 kg`, `p[2] = 70 kg`, `n[2] = 50 kg`.

Constraints for Type-2 are satisfied as follows:

- peaches at $70/180 \approx 38.9\%$ (below the 40% maximum)
- nectarines at $50/180 \approx 27.8\%$ (above the 20% minimum)
- apples at $\approx 33.3\%$.

Type-2 has the highest selling price per kg and contains all fruits without violating the percentage bounds. Allocating any kilogram to Type-1 (2.25/kg) or Type-3 (2.60/kg) would only reduce profit.

In [9]:
solver = pyo.SolverFactory("glpk")
solver.solve(model_basket, tee=False)

model_basket.display()

Model unknown

  Variables:
    y : Size=3, Index=types
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :  36.0 :  None : False : False : NonNegativeIntegers
          3 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    a : Size=3, Index=types
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : False : False : NonNegativeReals
          2 :     0 :  60.0 :  None : False : False : NonNegativeReals
          3 :     0 :   0.0 :  None : False : False : NonNegativeReals
    p : Size=3, Index=types
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : False : False : NonNegativeReals
          2 :     0 :  70.0 :  None : False : False : NonNegativeReals
          3 :     0 :   0.0 :  None : False : False : NonNegativeReals
    n : Size=3, Index=types
        Key : Lower : Value :

# Problem 2

A military base needs to deliver essential aid to three disaster areas (D1, D2, and D3) using a mixed fleet of four available transport aircraft (A1, A2, A3, and A4). The primary objective is to minimize the cost of the overall mission, without exceeding the aircraft capacity and assuming that each aircraft can perform several flights.

There is an amount of 350 tons of cargo to be delivered. The airplane characteristics is provided below:

| Aircraft | Cargo Capacity (tons) | Cost per Flight (USD)
| ---|---|---|
|A1|18|5000|
|A2|50|10000|
|A3|24|7000|
|A4|70|15000|

Following is the disaster area demands:

|Disaster Area | Cargo Required (tons) |
|---|---|
|D1|50|
|D2|200|
|D3|100|

Formulate an ILP model and solve it.

## 2.1. Objective function

The objective is to minimize the cost of the overall mission, defined as:

$$\min Z = \sum_{a}\sum_{d} c_a f_{a,d}$$

Where:

- $f_{a,d}$ is the number of flights from aircraft $a$ to disaster area $d$.
- $c_a$ is the cost of money per flight.

## 2.2. Constraints

Each area’s demand must be met and no aircraft can exceed its per-flight capacity:

- Per-area demand satisfaction, where $x_{a,d}$ is tons delivered by aircraft $a$ to area $j$:

$$\sum_a x_{a,d} = D_d \forall d$$

- Capacity per aircraft and area:

$$x_{a,d} \le c_a f_{a,d} \forall a,d$$

- Nonnegativity, integrality:

$$f_{a,d} \ge 0, \quad integer$$

## 2.3. PyOMO model

The lists `aircraft` and `areas` define the sets, while the dictionaries give each aircraft’s capacity (tons per flight) and flight cost, and each area’s demand.

In [ ]:
aircraft = ['A1','A2','A3','A4']
areas = ['D1','D2','D3']

capacity = {'A1':18,'A2':50,'A3':24,'A4':70}
cost = {'A1':5000,'A2':10000,'A3':7000,'A4':15000}
demand = {'D1':50,'D2':200,'D3':100}

A `ConcreteModel` is created to store information regarding the problem. The sets `A` and `D` are filled with the available aircraft and disaster areas.

Two types of decision variables are then declared. The integer variable `f` represents the number of flights performed by aircraft $a$ to area $d, while the continuous variable `x` captures the tons of cargo actually transported on that route.

In [11]:
model_aircraft = pyo.ConcreteModel()
model_aircraft.A = pyo.Set(initialize=aircraft)
model_aircraft.D = pyo.Set(initialize=areas)
model_aircraft.f = pyo.Var(model_aircraft.A, model_aircraft.D, domain=pyo.NonNegativeIntegers)
model_aircraft.x = pyo.Var(model_aircraft.A, model_aircraft.D, domain=pyo.NonNegativeReals)

The objective function sums the total cost of all flights across every aircraft–area combination, while trying to minimize this value. Each term `cost[a] * model_aircraft.f[a,d]` represents the fixed cost of sending a certain number of flights from aircraft $a$ to disaster area $d$.

In [12]:
model_aircraft.obj = pyo.Objective(expr=sum(cost[a]*model_aircraft.f[a,d] for a in model_aircraft.A for d in model_aircraft.D), sense=pyo.minimize)

As for the constraints, it needs to be ensured that each disaster area receives the exact amount of cargo required. The rule function `demand_rule()` sums all the tons of cargo `x` delivered to area `d` by every aircraft `a` and validates this total equal to the area’s demand.

The rule is applied to all disaster areas, and guarantees that the total cargo sent to each destination must precisely match the specified requirement.

In [13]:
def demand_rule(m,d):
    return sum(m.x[a,d] for a in m.A) == demand[d]

model_aircraft.demand = pyo.Constraint(model_aircraft.D, rule=demand_rule)

The function `capacity_rule()` limits the amount of cargo transported to the maximum that can be carried by the flights assigned. That means that, each flight of aircraft $a$ can carry up to its own capacity, and the total transported to destination $d$ cannot exceed the product of capacity and number of flights. By associating this rule with all aircraft and areas, the model ensures that cargo allocations remain feasible.

In [14]:
def capacity_rule(m,a,d):
    return m.x[a,d] <= capacity[a]*m.f[a,d]

model_aircraft.cap = pyo.Constraint(model_aircraft.A, model_aircraft.D, rule=capacity_rule)

## 2.4. Solving with GLPK

The solver assigns all shipments to Aircraft A2: `f[A2,D1]=1`, `f[A2,D2]=4`, `f[A2,D3]=2`, with a total of 7 flights. The delivered tons exactly match the demands with full loads on each route: `x[A2,D1]=50`, `x[A2,D2]=200`, `x[A2,D3]=100`, while all `f` and `x` for A1, A3, and A4 remain zero. The objective value is $7 \times 10000$ = \$70000.

All constraints were followed, and for A2 the capacity links are using the full capacity. As A2 has the lowest cost per ton when fully loaded ($200/ton), and the total requirement  of 350 tons is a multiple of A2’s capacity of 60 tons, no other aircraft can reduce cost under the given prices and capacities.

In [15]:
solver = pyo.SolverFactory('glpk')
solver.solve(model_aircraft)

model_aircraft.display()

Model unknown

  Variables:
    f : Size=12, Index=A*D
        Key          : Lower : Value : Upper : Fixed : Stale : Domain
        ('A1', 'D1') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('A1', 'D2') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('A1', 'D3') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('A2', 'D1') :     0 :   1.0 :  None : False : False : NonNegativeIntegers
        ('A2', 'D2') :     0 :   4.0 :  None : False : False : NonNegativeIntegers
        ('A2', 'D3') :     0 :   2.0 :  None : False : False : NonNegativeIntegers
        ('A3', 'D1') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('A3', 'D2') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('A3', 'D3') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('A4', 'D1') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('A4', 'D2') :     0 :   0.0 :  None 

# Problem 3

A large supermarket chain in the UK needs to build warehouses for a set of supermarkets it is opening in Northern England. The locations of the supermarkets have been identified, but the locations of the warehouses have yet to be determined.

Several good candidate locations for the warehouses have been identified, but decisions must be made regarding how many warehouses to open and at which candidate locations to build them. Moreover, each supermarket has an specific demand that must be met by the warehouses. If one warehouse is not sufficient to meet the demand, other warehouses can chosen to supply the supermarket.

Opening many warehouses would be advantageous as this would reduce the average distance a truck has to drive from the warehouse to the supermarket, and hence reduce the delivery cost. However, opening a warehouse has a fixed cost associated with it.

Our goal is **to find the optimal tradeoff between delivery costs and the costs of building new facilities**.

Below is the set of supermarkets. The coordinates and demand of each supermarket are provided in the following table.

| <i></i> | Coordinates | Demand |
| --- | --- | --- | 
| Supermarket 1 | (0,1.5) | 16,000 |
| Supermarket 2 | (2.5,1.2) | 18,000 |
| Supermarket 3 | (4,4) | 12,000 |
| Supermarket 3 | (1.0,1.3) | 17,000 |
| Supermarket 3 | (1.5,2.2) | 20,000 |

The following table shows the coordinates of the candidate warehouse sites, the fixed cost of building the warehouse in millions dollar, and its respective capacity.

| <i></i> | coordinates | fixed cost | Capacity |
| --- | --- |  --- | --- |
| Warehouse 1 | (0,0) | 3 | 30,000 |
| Warehouse 2 | (0,1) | 2 | 10,000 |
| Warehouse 3 | (0,2) | 3 | 12,000 |
| Warehouse 4 | (1,0) | 1 | 8,000 |
| Warehouse 5 | (1,1) | 3 | 15,000 |
| Warehouse 6 | (1,2) | 3 | 22,000 |
| Warehouse 7 | (2,0) | 4 | 10,000 |
| Warehouse 8 | (2,1) | 3 | 30,000 | 
| Warehouse 9 | (2,2) | 2 | 20,000 |

The transport cost is one million dollar per kilometer.

Formulate an ILP model and solve it.

## 3.1. Objective function

The problem involves two sets of elements: supermarkets and candidate warehouses. Each supermarket $i \in I$ is characterized by its geographic coordinates $(x_i, y_i)$ and a nonnegative demand $d_i$, measured in thousand units. Each warehouse $j \in J$ is defined by coordinates $(X_j, Y_j)$, a capacity $K_j$ also in thousand units, and a fixed opening cost $F_j$ expressed in millions of dollars.

The transportation cost between a warehouse and a supermarket depends on the Euclidean distance between their coordinates, calculated as $dist_{ij} = \sqrt{(x_i - X_j)^2 + (y_i - Y_j)^2}$, measured in kilometers. Transport costs are proportional to the distance traveled, so the shipping cost per thousand units is given by $c*{ij} = dist_{ij}$, in millions of dollars.

The objective is to minimize the total cost, combining fixed opening costs and transportation costs:

$$\min Z = \sum_j F_j y_j + \sum_i\sum_j c_{ij} x_{ij}$$

Where:

- $y_j$: binary variable indicating if warehouse $j$ is opened
- $x_{ij}$: thousands of units delivered from warehouse $j$ to supermarket $i$

## 3.2. Constraints

Each supermarket’s demand must be met and no warehouse can exceed its capacity:

- Demand satisfaction (allowing split deliveries):

$$\sum_j x_{ij} = d_i \forall i$$

- Demand satisfaction (allowing split deliveries):

$$\sum_i x_{ij} \leq K_j y_j \forall j$$

- Domains:

$$y_j \in \{0,1\}, \; x_{ij}\geq 0 \forall i,j$$

## 3.3. PyOMO model

## 3.4. Solving with GLPK